# First Contact with TensorFlow

 1. 텐서플로우 기본다지기
 2. 텐서플로우 선형회귀분석 
 3. 텐서플로우 클러스터링
 4. 텐서플로우 단일 레이어 뉴럴 네트워크
 5. 텐서플로우 다중 레이어 뉴럴 네트워크
 6. 병렬처리

## 1. 텐서플로우 기본다지기

In [ ]:
### 기본설정
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [ ]:
reset_graph()

a = tf.placeholder("float")
b = tf.placeholder("float")

y = tf.multiply(a, b)

sess = tf.Session()

print(sess.run(y, feed_dict={a: 3, b: 3}))
sess.close()

tensorflow operations  
  
tf.add  
tf.subtract  
tf.multiply  
tf.truediv  
tf.mod  
tf.abs  
tf.negative  
tf.sign  
tf.reciprocal ##역수 tf.reciprocal(3) = 1/3  
tf.square  
tf.round ##반올림 값  
tf.sqrt ##제곱근  
tf.pow ## 거듭제곱  
tf.exp ## 지수 값 y = e^x  
tf.log ## 로그 랎 y = ln e  
tf.maximum  
tf.minimum  
tf.cos  
tf.sin  

In [ ]:
reset_graph()

a = tf.diag([1,2,3,4,5])

sess = tf.Session()
print(sess.run(a))
sess.close()

In [ ]:
with tf.Session() as sess:
    t = a.eval()
t

행렬 연산  

tf.daig  
tf.transpose  
tf.matmul  
tf.matrix_determinant  
tf.matrix_inverse  

## 2. 텐서플로우 선형 회귀분석

In [ ]:
import numpy as np


num_points = 1000
vectors_set = []
for i in range(num_points):
    x1 = np.random.normal(0.0, 0.55)
    y1 = x1 * 0.1 + 0.3 + np.random.normal(0.0, 0.03)
    vectors_set.append([x1, y1])

x_data = [v[0] for v in vectors_set]
y_data = [v[1] for v in vectors_set]

import matplotlib.pyplot as plt

plt.plot(x_data, y_data, 'ro',label ='Original data', alpha=0.5)
plt.legend()

In [ ]:
reset_graph()

### 1 = 랜덤 숫자 개수
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))

y = W * x_data + b

## reduce_mean 차원을 줄이면서 계산함
## axis를 통해 축을 결정해줄 수 있음
loss = tf.reduce_mean(tf.square(y - y_data))

optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)


## 변수보다 아래에 정의되어야 함
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    for step in range(8):
        sess.run(train)
        print(step, W.eval(), b.eval())
        print(step, loss.eval())
        
        print(W.eval(), b.eval())
        plt.plot(x_data, y_data, 'ro')
        plt.plot(x_data, W.eval() * x_data + b.eval())
        plt.xlabel('x')
        plt.xlim(-2,2)
        plt.ylim(0.1,0.6)
        plt.ylabel('y')
    

## 3. 텐서플로우 클러스터링

tensor 타입에 관련된 함수  

tf.shape 
tf.size  
tf.rank  
tf.reshape  
tf.squeeze  
tf.expand_dims  
tf.slice  
tf.split  
tf.tile  
tf.concat  
tf.reverse  
tf.transpose  
tf.gather  


In [ ]:
reset_graph()

vectors = tf.constant([[1,2,3,4],[5,6,7,8]])
expanded_vectors = tf.expand_dims(vectors, 0)
##두번째 인자는 추가하는 차원의 위치
##2는 필요없는 이유가 텐서를 그냥 늘리면 되기 때문인듯
##ex tf.concat으로

print(expanded_vectors.get_shape())

상수를 생성할 수 있는 여러가지 도구들

tf.zeros_like  
tf.ones_like  
tf.fill  
tf.constant  


tf.random_normal 정규 분포 형태를 갖는 난수 텐서  
tf.truncated_normal 2표준편차 밖에 있는 값은 제외한 정규 분포 형태를 갖는 난수 텐서  
tf.random_uniform 균등 분포 형태의 난수 텐서를 생성  
tf.random_shuffle 첫번째 차원을 기준으로 하여 텐서의 엘리먼트를 섞음  
tf.set_random_seed 난수 시드를 제공  

In [ ]:
reset_graph()

vectors_ones = tf.zeros_like([1,2])

with tf.Session() as sess:
    print(vectors_ones.eval())

In [ ]:
reset_graph()

a = tf.placeholder("float")
b = tf.placeholder("float")
y = tf.multiply(a, b)

with tf.Session() as sess:
    print(y.eval(feed_dict={a: 3, b: 3}))

### kmeans clustering 코드따라가 보기

In [ ]:
reset_graph()

num_puntos = 2000
conjunto_puntos = []
for i in range(num_puntos):
    if np.random.random() > 0.5:
        conjunto_puntos.append([np.random.normal(0.0, 0.9),
                               np.random.normal(0.0, 0.9)])
    else:
        conjunto_puntos.append([np.random.normal(3.0, 0.5), np.random.normal(1.0, 0.5)])

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

df = pd.DataFrame({"x": [v[0] for v in conjunto_puntos],
                  "y": [v[1] for v in conjunto_puntos]})
sns.lmplot("x", "y", data=df, fit_reg=False, size=6)

In [ ]:
## kmeans 가자
vectors = tf.constant(conjunto_puntos)
k = 4
centroides = tf.Variable(tf.slice(tf.random_shuffle(vectors),[0,0],[k,-1]))
##centroides는 2차원 배열인데 이 것을 섞고 그 중 0,0 부터 3,1까지 가져옴


In [ ]:
expanded_vectors = tf.expand_dims(vectors, 0) ##2000개
expanded_centroides = tf.expand_dims(centroides, 1) ## 4개
##브로드캐스팅을 위해서

diff = tf.subtract(expanded_vectors, expanded_centroides)
sqr = tf.square(diff)
distances = tf.reduce_sum(sqr, 2) ## 2는 axis
assignments = tf.argmin(distances, 0) ## 0는 axis

tf.reduce_sum처럼 텐서의 차원을 감소시키는 수학연산들  
tf.reduce_sum  
tf.reduce_prod  
tf.reduce_min  
tf.reduce_max  
tf.reduce_mean  

가장 작거나 큰 값의 인덱스를 리턴하는 함수

tf.argmin  
tf.argmax  

In [ ]:
print(expanded_centroides.get_shape())
print(expanded_vectors.get_shape())

In [ ]:
means = tf.concat([tf.reduce_mean(tf.gather(vectors, tf.reshape(tf.where(tf.equal(assignments, c)),
                                                                [1, -1])), reduction_indices=[1]) for c in range(k)], 0)

In [ ]:
updata_centroides = tf.assign(centroides, means)

In [ ]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    for step in range(100):
        _, centroid_values, assignment_values = updata_centroides.eval(), centroides.eval(), assignments.eval()

In [ ]:
print(centroid_values)

In [ ]:
data = {"x": [], "y": [], "cluster": []}
for i in range(len(assignment_values)):
    data["x"].append(conjunto_puntos[i][0])
    data["y"].append(conjunto_puntos[i][1])
    data["cluster"].append(assignment_values[i])
df = pd.DataFrame(data)
sns.lmplot("x", "y", data=df, 
           fit_reg=False, size=7, 
           hue="cluster", legend=False)

## 4. 텐서플로우 단일 레이어 뉴럴 네트워크

In [ ]:
reset_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
tf.convert_to_tensor(mnist.train.images).get_shape()

In [ ]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

x = tf.placeholder("float", [None, 784])
y = tf.nn.softmax(tf.matmul(x,W) + b) ## 예측값의 소프트맥스

In [ ]:
y_ = tf.placeholder("float", [None, 10])
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [ ]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    for i in  range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        train_step.run(feed_dict={x: batch_xs, y_: batch_ys})
        correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        if i % 10 == 0: 
            print(accuracy.eval(feed_dict={x: mnist.test.images, y_:mnist.test.labels}))